In [2]:
# --- CELL 1: MASTER SETUP (KAGGLE) ---

# 1. Install YOLOv8
# (Make sure "Internet" is enabled in your Kaggle notebook settings)
!pip install -q ultralytics

# 2. Check for the T4 GPU
print("--- Checking GPU ---")
!nvidia-smi -L
print("--------------------")

# 3. Import ALL necessary libraries
import os
import zipfile
import shutil
import glob
from tqdm.auto import tqdm
from ultralytics import YOLO
from PIL import Image  # This is for opening/reading images
# (IPython.display and Google Drive imports removed)

print("✅ Libraries installed and imported correctly.")

# 4. (Google Drive mount removed)

# 5. Define base paths
# Replace '<your-dataset-name>' with the name of your dataset
input_data_path = '/kaggle/input/visdrone'
output_working_path = '/kaggle/working/'

print(f"Data will be read from: {input_data_path}")
print(f"Output files will be saved to: {output_working_path}")
print("✅ Paths defined.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [3]:
import zipfile
import os

# --- Define Zip File Paths ---
# Assumes 'input_data_path' was defined in the previous cell
train_zip_path = os.path.join(input_data_path, 'modified_visdrone_scaled_annos.zip')
val_zip_path = os.path.join(input_data_path, 'modified_visdrone_scaled_annos_val.zip')

# --- Define Unzip Destinations ---
base_unzip_dir = '/kaggle/working/visdrone_data'
train_dest = os.path.join(base_unzip_dir, 'train_data')
val_dest = os.path.join(base_unzip_dir, 'val_data')

# Create the base directory if it doesn't exist
os.makedirs(base_unzip_dir, exist_ok=True)

# --- Unzip Training Data ---
print(f"Unzipping {train_zip_path}...")
if os.path.exists(train_zip_path):
    with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
        zip_ref.extractall(base_unzip_dir)
        
        extracted_folder = os.path.join(base_unzip_dir, 'modified_visdrone_scaled_annos')
        if os.path.exists(extracted_folder):
            os.rename(extracted_folder, train_dest)
            print("Training data unzipped.")
        else:
            print(f"Warning: Expected folder '{extracted_folder}' not found after extraction.")
else:
    print(f"❌ ERROR: {train_zip_path} not found!")

# --- Unzip Validation Data ---
print(f"Unzipping {val_zip_path}...")
if os.path.exists(val_zip_path):
    with zipfile.ZipFile(val_zip_path, 'r') as zip_ref:
        zip_ref.extractall(base_unzip_dir)
        
        extracted_folder_val = os.path.join(base_unzip_dir, 'modified_visdrone_scaled_annos_val')
        if os.path.exists(extracted_folder_val):
            os.rename(extracted_folder_val, val_dest)
            print("Validation data unzipped.")
        else:
             print(f"Warning: Expected folder '{extracted_folder_val}' not found after extraction.")
else:
    print(f"❌ ERROR: {val_zip_path} not found!")

print("\n✅ Datasets unzipped to /kaggle/working/visdrone_data")

/kaggle/input/visdrone/modified_visdrone_scaled_annos

Unzipping /kaggle/input/visdrone/modified_visdrone_scaled_annos.zip...
❌ ERROR: /kaggle/input/visdrone/modified_visdrone_scaled_annos.zip not found!
Unzipping /kaggle/input/visdrone/modified_visdrone_scaled_annos_val.zip...
❌ ERROR: /kaggle/input/visdrone/modified_visdrone_scaled_annos_val.zip not found!

✅ Datasets unzipped to /kaggle/working/visdrone_data


NameError: name 'kaggle' is not defined

In [4]:
# --- CELL 3: CONVERT ANNOTATIONS (KAGGLE) ---
import json
import glob
import shutil
from tqdm.auto import tqdm
from PIL import Image

# --- 1. Define VisDrone categories ---
# These are the 10 classes we care about, matching the original script's logic
# (skipping 0: 'ignored' and 11: 'other')
VISDRONE_CATEGORIES = [
    {"id": 1, "name": "pedestrian", "supercategory": "person"},
    {"id": 2, "name": "person", "supercategory": "person"},
    {"id": 3, "name": "bicycle", "supercategory": "vehicle"},
    {"id": 4, "name": "car", "supercategory": "vehicle"},
    {"id": 5, "name": "van", "supercategory": "vehicle"},
    {"id": 6, "name": "truck", "supercategory": "vehicle"},
    {"id": 7, "name": "tricycle", "supercategory": "vehicle"},
    {"id": 8, "name": "awning-tricycle", "supercategory": "vehicle"},
    {"id": 9, "name": "bus", "supercategory": "vehicle"},
    {"id": 10, "name": "motor", "supercategory": "vehicle"},
]

# --- 2. Define Source and Destination Paths ---
# Data was unzipped to /kaggle/working/ in the previous cell
visdrone_train_dir = '/kaggle/input/visdrone/modified_visdrone_scaled_annos/modified_visdrone_scaled_annos'
visdrone_val_dir = '/kaggle/input/visdrone/modified_visdrone_scaled_annos_val/modified_visdrone_scaled_annos_val'
# All new files must be written to /kaggle/working/
coco_base_dir = '/kaggle/working/coco_dataset'

# --- 3. Create the COCO Directory Structure ---
# This structure is common for COCO datasets
os.makedirs(os.path.join(coco_base_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(coco_base_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(coco_base_dir, 'annotations'), exist_ok=True)

print("COCO directory structure created in /kaggle/working/.")



# --- 4. Define the Conversion Function ---
def convert_visdrone_to_coco(source_dir, dest_base_dir, split, categories):
    """
    Converts VisDrone annotations to COCO format (for Faster R-CNN).

    Args:
        source_dir (str): Path to the source VisDrone data (e.g., '/kaggle/working/visdrone_data/train_data')
        dest_base_dir (str): Path to the root of the destination COCO dataset (e.g., '/kaggle/working/coco_dataset')
        split (str): 'train' or 'val'
        categories (list): The list of category dictionaries.
    """
    source_images_dir = os.path.join(source_dir, 'images')
    source_annos_dir = os.path.join(source_dir, 'annotations')

    dest_images_dir = os.path.join(dest_base_dir, 'images', split)

    # Initialize the main COCO data structure
    coco_data = {
        "images": [],
        "annotations": [],
        "categories": categories
    }

    # Keep track of annotation IDs
    annotation_id_counter = 1

    anno_files = glob.glob(os.path.join(source_annos_dir, '*.txt'))
    print(f"\nConverting {split} data ({len(anno_files)} files) to COCO format...")

    # Use enumerate to create a unique, 1-based image_id for COCO
    for image_id, anno_file in enumerate(tqdm(anno_files), 1):
        base_filename = os.path.basename(anno_file)
        img_filename = os.path.splitext(base_filename)[0] + '.jpg'
        img_path = os.path.join(source_images_dir, img_filename)

        # 1. Copy the image
        if os.path.exists(img_path):
            shutil.copy(img_path, os.path.join(dest_images_dir, img_filename))
        else:
            print(f"Warning: Image not found for {base_filename}, skipping.")
            continue

        # 2. Read image dimensions
        try:
            with Image.open(img_path) as img:
                img_w, img_h = img.size
        except Exception as e:
            print(f"Error opening image {img_path}: {e}. Skipping.")
            continue

        # 3. Add image info to COCO data
        coco_data["images"].append({
            "id": image_id,
            "file_name": img_filename, # Just the filename
            "width": img_w,
            "height": img_h
        })

        # 4. Read VisDrone annotations and convert to COCO annotations
        with open(anno_file, 'r') as f:
            for line in f:
                parts = line.strip().split(',')

                # VisDrone class (1-based)
                vis_class = int(parts[5])

                # Skip classes 0 ('ignored') and 11 ('other') as per original logic
                if vis_class in [0, 11]:
                    continue

                # This is the category_id for COCO, which we've defined from 1-10
                category_id = vis_class

                # Read VisDrone bounding box (unnormalized)
                x_min = int(parts[0])
                y_min = int(parts[1])
                w = int(parts[2])
                h = int(parts[3])

                # COCO format requires [x_min, y_min, width, height]
                bbox = [x_min, y_min, w, h]

                # COCO format also requires area
                area = w * h

                # Add the annotation
                coco_data["annotations"].append({
                    "id": annotation_id_counter,
                    "image_id": image_id,
                    "category_id": category_id,
                    "bbox": bbox,
                    "area": area,
                    "iscrowd": 0, # Assuming not a crowd object
                    "segmentation": [] # Required, but can be empty for bbox
                })

                annotation_id_counter += 1

    # 5. Write the final COCO JSON annotation file
    json_path = os.path.join(dest_base_dir, 'annotations', f'instances_{split}.json')

    with open(json_path, 'w') as f_out:
        json.dump(coco_data, f_out, indent=4)

# --- 5. Run the conversion for both splits ---
convert_visdrone_to_coco(visdrone_train_dir, coco_base_dir, 'train', VISDRONE_CATEGORIES)
convert_visdrone_to_coco(visdrone_val_dir, coco_base_dir, 'val', VISDRONE_CATEGORIES)

print("\n✅ Annotation conversion to COCO format complete.")
print(f"Your dataset is ready in: {coco_base_dir}")
print(f"Training annotations: {coco_base_dir}/annotations/instances_train.json")
print(f"Validation annotations: {coco_base_dir}/annotations/instances_val.json")

COCO directory structure created in /kaggle/working/.

Converting train data (6471 files) to COCO format...


  0%|          | 0/6471 [00:00<?, ?it/s]


Converting val data (548 files) to COCO format...


  0%|          | 0/548 [00:00<?, ?it/s]


✅ Annotation conversion to COCO format complete.
Your dataset is ready in: /kaggle/working/coco_dataset
Training annotations: /kaggle/working/coco_dataset/annotations/instances_train.json
Validation annotations: /kaggle/working/coco_dataset/annotations/instances_val.json


In [5]:
import os
import torch
import torchvision
# --- CHANGE 1: Import MobileNet model and weights ---
from torchvision.models.detection import (
    fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights,
    fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import v2 as T
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm

# --- 1. Custom Dataset for COCO ---
# We need this to correctly format the targets (annotations) for the model
class CustomCocoDataset(torchvision.datasets.CocoDetection):
    def __init__(self, root, annFile, transform=None):
        super(CustomCocoDataset, self).__init__(root, annFile)
        self.transform = transform

    def __getitem__(self, idx):
        # Load the raw PIL image and annotation list
        img, target_list = super(CustomCocoDataset, self).__getitem__(idx)

        # 1. Convert PIL Image to Tensor
        if self.transform is not None:
            img = self.transform(img)

        # 2. Convert annotation list to the required tensor dictionary
        target = {}
        image_id = torch.tensor([self.ids[idx]]) # Get the original COCO image_id

        # Handle images with no annotations
        if not target_list:
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros(0, dtype=torch.int64)
            target["image_id"] = image_id
            target["area"] = torch.zeros(0, dtype=torch.float32)
            target["iscrowd"] = torch.zeros(0, dtype=torch.int64)
            return img, target

        # Parse all annotations for this image
        boxes = [obj['bbox'] for obj in target_list]
        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
        # Convert [x, y, w, h] to [x1, y1, x2, y2]
        boxes[:, 2:] += boxes[:, :2]

        labels = [obj['category_id'] for obj in target_list]
        labels = torch.as_tensor(labels, dtype=torch.int64)

        area = torch.as_tensor([obj['area'] for obj in target_list], dtype=torch.float32)
        iscrowd = torch.as_tensor([obj['iscrowd'] for obj in target_list], dtype=torch.int64)

        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return img, target

# --- 2. Collate Function ---
# This is necessary to batch images and targets correctly
def collate_fn(batch):
    return tuple(zip(*batch))

# --- 3. Model Definition ---
def get_faster_rcnn_model(num_classes):
    """
    Loads a pre-trained Faster R-CNN model and replaces the classifier head.
    """
    # --- CHANGE 2: Use the lighter MobileNetV3 backbone ---
    # Load a model pre-trained on COCO
    # model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT) # <-- Original ResNet-50
    model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT) # <-- New MobileNet
    print("Using lightweight MobileNetV3-Large backbone.")

    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Replace the pre-trained head with a new one
    # num_classes includes your 10 classes + 1 background class
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

# --- 4. Main Training Script ---
def main():
    # --- Parameters ---
    # --- KAGGLE CHANGE ---
    # These paths come from your 'convert_to_coco.py' script, now in /kaggle/working/
    TRAIN_IMG_DIR = '/kaggle/working/coco_dataset/images/train'
    TRAIN_ANN_FILE = '/kaggle/working/coco_dataset/annotations/instances_train.json'

    # We will use 10 classes + 1 background class = 11
    NUM_CLASSES = 11

    # Parameters from your YOLO script
    EPOCHS = 20

    # --- CHANGE 3: Restore BATCH_SIZE (model is lighter) ---
    # The new lightweight model and smaller image size should allow for a larger batch.
    BATCH_SIZE = 16 # Back to original 16
    ACCUMULATION_STEPS = 1 # 1 means no accumulation
    print(f"Using BATCH_SIZE={BATCH_SIZE} with ACCUMULATION_STEPS={ACCUMULATION_STEPS} (effective batch size: {BATCH_SIZE * ACCUMULATION_STEPS})")

    # Project paths from your YOLO script
    # --- KAGGLE CHANGE ---
    # All outputs must be saved to /kaggle/working/
    PROJECT_DIR = '/kaggle/working/runs/detect'
    NAME = 'finetune_visdrone_night_scaled_FASTER_RCNN' # New name for new model
    OUTPUT_DIR = os.path.join(PROJECT_DIR, NAME)
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Set device (GPU or CPU)
    device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    print(f"Using device: {device}")

    # --- Dataset and DataLoader ---

    # --- CHANGE 4: Add Image Resizing (matches YOLO's imgsz=256) ---
    # This will dramatically speed up training and reduce memory.
    print("Applying transform: Resizing all images to 256x256.")
    dataset_train = CustomCocoDataset(
        root=TRAIN_IMG_DIR,
        annFile=TRAIN_ANN_FILE,
        transform=T.Compose([
            T.ToImage(),
            T.Resize((256, 256), antialias=True), # <-- Add resize
            T.ToDtype(torch.float32, scale=True)
        ])
    )

    data_loader_train = DataLoader(
        dataset_train,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        collate_fn=collate_fn
    )

    # --- Model, Optimizer, Scheduler ---
    print("Loading Faster R-CNN model...")
    model = get_faster_rcnn_model(NUM_CLASSES)
    model.to(device)

    # Optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    # Learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # --- Check for pre-trained weights (Optional) ---
    # This script fine-tunes from the default ResNet50 backbone.
    # If you had a 'best.pt' for Faster R-CNN, you would load it here.
    # We are starting from the public pre-trained model instead.
    print(f"✅ Model loaded and ready for fine-tuning.")
    print("Starting fine-tuning on T4 GPU...")

    # --- Training Loop ---
    for epoch in range(EPOCHS):
        model.train()
        epoch_loss = 0

        # Zero the gradients once at the start of the epoch
        optimizer.zero_grad()

        # Use tqdm for a nice progress bar
        # loop = tqdm(data_loader_train, desc=f"Epoch {epoch+1}/{EPOCHS} [Training]")
        loop = tqdm(enumerate(data_loader_train), desc=f"Epoch {epoch+1}/{EPOCHS} [Training]", total=len(data_loader_train))

        for i, (images, targets) in loop:
            # Move data to the correct device
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Forward pass (returns a dict of losses in train mode)
            loss_dict = model(images, targets)

            # Sum all losses
            losses = sum(loss for loss in loss_dict.values())

            # --- Gradient Accumulation ---
            # (No longer needed, ACCUMULATION_STEPS = 1)
            losses = losses / ACCUMULATION_STEPS
            losses.backward()
            if (i + 1) % ACCUMULATION_STEPS == 0:
                optimizer.step()
                optimizer.zero_grad()
            # --- End Gradient Accumulation ---
            epoch_loss += losses.item() * ACCUMULATION_STEPS
            loop.set_postfix(loss=losses.item()) # Show the normalized loss

        # --- End of Epoch ---
        if (len(data_loader_train) % ACCUMULATION_STEPS) != 0:
            optimizer.step()
            optimizer.zero_grad()

        # Step the learning rate scheduler
        lr_scheduler.step()

        # avg_loss = epoch_loss / len(data_loader_train)
        avg_loss = epoch_loss / len(data_loader_train.dataset) # More accurate loss
        print(f"\nEpoch {epoch+1}/{EPOCHS} - Average Training Loss: {avg_loss:.4f}")

        # Save a checkpoint
        if (epoch + 1) % 10 == 0 or (epoch + 1) == EPOCHS:
            checkpoint_path = os.path.join(OUTPUT_DIR, f'model_epoch_{epoch+1}.pt')
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Saved checkpoint: {checkpoint_path}")

    print("✅ Training complete.")

    final_model_path = os.path.join(OUTPUT_DIR, 'faster_rcnn_final.pt')
    torch.save(model.state_dict(), final_model_path) # <-- CORRECTED
    print(f"Final model saved to: {final_model_path}")

# Run the training
if __name__ == "__main__":
    main()

Using BATCH_SIZE=16 with ACCUMULATION_STEPS=1 (effective batch size: 16)
Using device: cuda:0
Applying transform: Resizing all images to 256x256.
loading annotations into memory...
Done (t=2.03s)
creating index...
index created!
Loading Faster R-CNN model...


Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth
100%|██████████| 74.2M/74.2M [00:00<00:00, 153MB/s] 


Using lightweight MobileNetV3-Large backbone.
✅ Model loaded and ready for fine-tuning.
Starting fine-tuning on T4 GPU...


Epoch 1/20 [Training]: 100%|██████████| 405/405 [05:04<00:00,  1.33it/s, loss=1.24]


Epoch 1/20 - Average Training Loss: 0.0876


...


Epoch 18/20 - Average Training Loss: 0.0696



Epoch 19/20 [Training]: 100%|██████████| 405/405 [05:08<00:00,  1.31it/s, loss=1.09] 


Epoch 19/20 - Average Training Loss: 0.0694



Epoch 20/20 [Training]: 100%|██████████| 405/405 [05:08<00:00,  1.31it/s, loss=0.997]



Epoch 20/20 - Average Training Loss: 0.0694
Saved checkpoint: /kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN/model_epoch_20.pt
✅ Training complete.
Final model saved to: /kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN/faster_rcnn_final.pt


In [14]:
# --- CELL: EVALUATE mAP ON VALIDATION SET ---

# 1. Install/Upgrade torchmetrics, scipy, and numpy to fix the ValueError
!pip install -q --upgrade numpy scipy torchmetrics

# --- 🔴 STOP! 🔴 ---
# After this pip install finishes, you MUST RESTART THE KERNEL.
# In the Kaggle menu: Click "Run" -> "Restart Session"
# After restarting, run this *ENTIRE CELL* again.
# --------------------

import os
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import v2 as T
from tqdm import tqdm

# This import will only work AFTER you restart the session
from torchmetrics.detection import MeanAveragePrecision

# --- KAGGLE CHANGE: Add imports and function needed to build the model ---
from torchvision.models.detection import (
    fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# --- 2. Redefine Dataset and Model Functions ---
# We must redefine these so this cell can run independently

class CustomCocoDataset(torchvision.datasets.CocoDetection):
    def __init__(self, root, annFile, transform=None):
        super(CustomCocoDataset, self).__init__(root, annFile)
        self.transform = transform

    def __getitem__(self, idx):
        img, target_list = super(CustomCocoDataset, self).__getitem__(idx)
        
        # 1. Convert PIL Image to Tensor
        if self.transform is not None:
            img = self.transform(img)

        # 2. Convert annotation list to the required tensor dictionary
        target = {}
        image_id = torch.tensor([self.ids[idx]])

        if not target_list:
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros(0, dtype=torch.int64)
            target["image_id"] = image_id
            target["area"] = torch.zeros(0, dtype=torch.float32)
            target["iscrowd"] = torch.zeros(0, dtype=torch.int64)
            return img, target

        boxes = [obj['bbox'] for obj in target_list]
        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
        # Convert [x, y, w, h] to [x1, y1, x2, y2]
        boxes[:, 2:] += boxes[:, :2]

        labels = [obj['category_id'] for obj in target_list]
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = torch.as_tensor([obj['area'] for obj in target_list], dtype=torch.float32)
        iscrowd = torch.as_tensor([obj['iscrowd'] for obj in target_list], dtype=torch.int64)

        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return img, target

def get_faster_rcnn_model(num_classes):
    model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def collate_fn(batch):
    return tuple(zip(*batch))

# --- 3. Setup Paths and Parameters ---
MODEL_PATH = '/kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN/faster_rcnn_final.pt'
VAL_IMG_DIR = '/kaggle/working/coco_dataset/images/val'
VAL_ANN_FILE = '/kaggle/working/coco_dataset/annotations/instances_val.json'
NUM_CLASSES = 11
BATCH_SIZE = 16
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

print(f"Using device: {device}")

# --- 4. Load Validation Data ---
# CRITICAL: The transform must match the training transform, including resize!
val_transform = T.Compose([
    T.ToImage(),
    T.Resize((256, 256), antialias=True),
    T.ToDtype(torch.float32, scale=True)
])

dataset_val = CustomCocoDataset(
    root=VAL_IMG_DIR,
    annFile=VAL_ANN_FILE,
    transform=val_transform
)

data_loader_val = DataLoader(
    dataset_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

# --- 5. Load Model ---
if not os.path.exists(MODEL_PATH):
    print(f"❌ ERROR: Model file not found at {MODEL_PATH}")
else:
    print("Loading model for evaluation...")
    model = get_faster_rcnn_model(NUM_CLASSES)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.to(device)
    model.eval()
    print("Model loaded.")

    # --- 6. Initialize Metric ---
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.to(device)

    # --- 7. Evaluation Loop ---
    print(f"Running evaluation on {len(dataset_val)} validation images...")
    
    with torch.no_grad():
        for images, targets in tqdm(data_loader_val, desc="Evaluating"):
            # Move data to device
            images = list(img.to(device) for img in images)
            
            # Get model predictions
            # Note: Model output is already in the format torchmetrics expects
            predictions = model(images) 
            
            # Format targets for torchmetrics
            # (must be on CPU for this, then moved to device)
            targets_formatted = []
            for t in targets:
                targets_formatted.append({
                    "boxes": t["boxes"].to(device),
                    "labels": t["labels"].to(device)
                })

            # Update the metric
            metric.update(predictions, targets_formatted)

    # --- 8. Compute and Print Final mAP ---
    print("\nComputing final mAP scores...")
    results = metric.compute()
    
    print("\n--- FASTER R-CNN mAP RESULTS ---")
    print(f"mAP (IoU=0.50:0.95): {results['map']:.4f}")
    print(f"mAP (IoU=0.50):     {results['map_50']:.4f}")
    print(f"mAP (IoU=0.75):     {results['map_75']:.4f}")
    print("----------------------------------")
    print(f"mAP (Small):        {results['map_small']:.4f}")
    print(f"mAP (Medium):       {results['map_medium']:.4f}")
    print(f"mAP (Large):        {results['map_large']:.4f}")
    print("----------------------------------")

ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

In [6]:
import os

print("--- Training Results (Faster R-CNN) ---")

# --- KAGGLE CHANGE ---
# This path matches the 'OUTPUT_DIR' from your train_faster_rcnn.py script
results_dir = '/kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN'

if os.path.exists(results_dir):
    print(f"Results are saved in: {results_dir}")

    # List all files in the directory (checkpoints and final model)
    !ls -lh {results_dir}

    print("\n--- Training Metrics ---")
    print("Training loss was printed to the console after each epoch.")
    print("Note: A 'results.png' is not automatically generated for this type of training.")
    print("To get validation mAP, a separate evaluation script must be run.")

    # Point to the final model
    final_model_path = os.path.join(results_dir, 'faster_rcnn_final.pt')
    if os.path.exists(final_model_path):
        print(f"\nYour final fine-tuned model is at: {final_model_path}")
    else:
        print("\nFinal model file 'faster_rcnn_final.pt' not found. Training may still be running or encountered an error.")
else:
    print(f"❌ ERROR: Results directory not found at {results_dir}")
    print("Please ensure the 'train_faster_rcnn.py' script ran successfully.")

--- Training Results (Faster R-CNN) ---
Results are saved in: /kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN
total 219M
-rw-r--r-- 1 root root 73M Nov 17 12:33 faster_rcnn_final.pt
-rw-r--r-- 1 root root 73M Nov 17 11:41 model_epoch_10.pt
-rw-r--r-- 1 root root 73M Nov 17 12:33 model_epoch_20.pt

--- Training Metrics ---
Training loss was printed to the console after each epoch.
Note: A 'results.png' is not automatically generated for this type of training.
To get validation mAP, a separate evaluation script must be run.

Your final fine-tuned model is at: /kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN/faster_rcnn_final.pt


In [7]:
import os
import torch
import glob
from PIL import Image, ImageDraw
from torchvision.transforms import v2 as T
# from IPython.display import Image as IpyImage, display # Not needed in Kaggle script

# --- KAGGLE CHANGE: Add imports and function needed to build the model ---
from torchvision.models.detection import (
    fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_faster_rcnn_model(num_classes):
    """
    Loads a pre-trained Faster R-CNN model and replaces the classifier head.
    """
    model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)
    print("Using lightweight MobileNetV3-Large backbone.")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model
# --- End of added function ---


# --- 1. Load your newly fine-tuned model ---
# --- KAGGLE CHANGE ---
MODEL_DIR = '/kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN'
MODEL_PATH = os.path.join(MODEL_DIR, 'faster_rcnn_final.pt')
NUM_CLASSES = 11 # 10 VisDrone classes + 1 background
CONF_THRESHOLD = 0.25

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

if not os.path.exists(MODEL_PATH):
    print(f"❌ ERROR: Model file not found at {MODEL_PATH}")
    print("Please ensure the training cell ran successfully.")
else:
    print(f"Loading model from: {MODEL_PATH}")
    model = get_faster_rcnn_model(NUM_CLASSES)
    if model:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        model.to(device)
        model.eval() # Set model to evaluation mode

        # --- 2. Get a random image from your validation set ---
        # --- KAGGLE CHANGE ---
        val_images = glob.glob('/kaggle/working/coco_dataset/images/val/*.jpg')
        if val_images:
            test_image_path = val_images[0]
            print(f"Running inference on: {test_image_path}")

            # Load the image
            img_pil = Image.open(test_image_path).convert('RGB')

            # --- KAGGLE CHANGE: CRITICAL FIX ---
            # The transform MUST match the training transform, including the resize.
            transform = T.Compose([
                T.ToImage(),
                T.Resize((256, 256), antialias=True), # <-- This was missing
                T.ToDtype(torch.float32, scale=True)
            ])
            img_tensor = transform(img_pil)

            # Add a batch dimension and send to device
            input_batch = [img_tensor.to(device)]

            # --- 3. Run prediction ---
            with torch.no_grad():
                predictions = model(input_batch)

            # predictions[0] contains the results for our single image
            pred = predictions[0]

            # --- 4. Draw results and display ---
            print("\n--- Detection Result (Boxes Only) ---")

            # Get boxes and scores that meet the confidence threshold
            high_conf_indices = pred['scores'] > CONF_THRESHOLD
            boxes = pred['boxes'][high_conf_indices].cpu().numpy()
            
            # --- KAGGLE CHANGE: Draw on the *original* un-resized image ---
            # We must scale the 256x256 boxes back to the original image size
            orig_w, orig_h = img_pil.size
            # The model output boxes are for a 256x256 image
            scale_x = orig_w / 256.0
            scale_y = orig_h / 256.0
            
            scaled_boxes = []
            for box in boxes:
                x1, y1, x2, y2 = box
                scaled_boxes.append([
                    x1 * scale_x, 
                    y1 * scale_y, 
                    x2 * scale_x, 
                    y2 * scale_y
                ])

            # Draw the *scaled* boxes on the *original* PIL image
            draw = ImageDraw.Draw(img_pil)
            for box in scaled_boxes:
                # 'box' is [x1, y1, x2, y2]
                draw.rectangle(list(box), outline="red", width=3)

            # Save the resulting image
            base_filename = os.path.basename(test_image_path)
            save_path = os.path.join(MODEL_DIR, f"detection_{base_filename}")
            img_pil.save(save_path)

            print(f"Result saved to: {save_path}")
            print("You can view the saved image in the Kaggle output viewer.")

            # --- KAGGLE CHANGE ---
            # Removed 'display(IpyImage(filename=save_path))'
            # Kaggle will automatically show images saved to /kaggle/working/

        else:
            print(f"No validation images found in /kaggle/working/coco_dataset/images/val/")
    else:
        print("Model could not be loaded. Skipping inference.")

Loading model from: /kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN/faster_rcnn_final.pt
Using lightweight MobileNetV3-Large backbone.
Running inference on: /kaggle/working/coco_dataset/images/val/0000287_02801_d_0000773.jpg

--- Detection Result (Boxes Only) ---
Result saved to: /kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN/detection_0000287_02801_d_0000773.jpg
You can view the saved image in the Kaggle output viewer.


In [8]:
import os
import torch
import glob
from PIL import Image, ImageDraw, ImageFont
from torchvision.transforms import v2 as T
from tqdm import tqdm

# --- KAGGLE CHANGE: Add imports and function needed to build the model ---
from torchvision.models.detection import (
    fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_faster_rcnn_model(num_classes):
    """
    Loads a pre-trained Faster R-CNN model and replaces the classifier head.
    """
    model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)
    # print("Using lightweight MobileNetV3-Large backbone.") # Silenced for less output
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model
# --- End of added function ---


# We need the class names from our 'convert_to_coco.py' script to draw labels
VISDRONE_CATEGORIES = [
    {"id": 1, "name": "pedestrian"}, {"id": 2, "name": "person"},
    {"id": 3, "name": "bicycle"}, {"id": 4, "name": "car"},
    {"id": 5, "name": "van"}, {"id": 6, "name": "truck"},
    {"id": 7, "name": "tricycle"}, {"id": 8, "name": "awning-tricycle"},
    {"id": 9, "name": "bus"}, {"id": 10, "name": "motor"},
]
# Create a mapping from ID (1-10) to name
COCO_CATEGORY_MAP = {cat['id']: cat['name'] for cat in VISDRONE_CATEGORIES}


# --- 2. Main Inference Function ---
def run_inference_on_all_val():
    print("Loading fine-tuned model...")
    # --- Paths and Parameters ---
    # --- KAGGLE CHANGE ---
    MODEL_DIR = '/kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN'
    MODEL_PATH = os.path.join(MODEL_DIR, 'faster_rcnn_final.pt')
    NUM_CLASSES = 11 # 10 VisDrone classes + 1 background

    # Equivalent to your YOLO script's parameters
    CONF_THRESHOLD = 0.25
    # --- KAGGLE CHANGE ---
    VAL_IMAGES_PATH = '/kaggle/working/coco_dataset/images/val/' # Use the COCO dataset path

    # Equivalent to project='runs/detect', name='predictions_with_labels'
    # --- KAGGLE CHANGE ---
    OUTPUT_DIR = '/kaggle/working/runs/detect/predictions_with_labels_FASTER_RCNN'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

    if not os.path.exists(MODEL_PATH) or not get_faster_rcnn_model:
        print(f"❌ ERROR: Model file not found at {MODEL_PATH} or 'get_faster_rcnn_model' not available.")
        print("Please ensure the training cell ran successfully.")
        return

    # --- Load Model ---
    model = get_faster_rcnn_model(NUM_CLASSES)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.to(device)
    model.eval() # Set model to evaluation mode
    print("Model loaded.")

    # --- Load Font for drawing labels ---
    try:
        # Use a default font that's available in PIL
        font = ImageFont.load_default()
    except IOError:
        print("Cannot load default font. Using dummy font.")
        font = None

    # --- Get all image paths ---
    val_images_paths = glob.glob(os.path.join(VAL_IMAGES_PATH, '*.jpg'))
    if not val_images_paths:
        print(f"No images found in {VAL_IMAGES_PATH}")
        return

    print(f"Running predictions on all {len(val_images_paths)} images in: {VAL_IMAGES_PATH}")

    # --- KAGGLE CHANGE: CRITICAL FIX ---
    # The transform MUST match the training transform, including the resize.
    transform = T.Compose([
        T.ToImage(),
        T.Resize((256, 256), antialias=True), # <-- This was missing
        T.ToDtype(torch.float32, scale=True)
    ])

    # --- Loop and predict (equivalent to model.predict) ---
    for img_path in tqdm(val_images_paths):
        # Load and transform the image
        img_pil = Image.open(img_path).convert('RGB')
        
        # --- KAGGLE CHANGE: Store original size for scaling boxes ---
        orig_w, orig_h = img_pil.size
        
        img_tensor = transform(img_pil) # This creates a 256x256 tensor
        input_batch = [img_tensor.to(device)]

        # Run prediction
        with torch.no_grad():
            predictions = model(input_batch)

        pred = predictions[0]

        # Filter boxes, scores, and labels by confidence
        high_conf_indices = pred['scores'] > CONF_THRESHOLD
        boxes = pred['boxes'][high_conf_indices].cpu().numpy()
        labels = pred['labels'][high_conf_indices].cpu().numpy()
        scores = pred['scores'][high_conf_indices].cpu().numpy()
        
        # --- KAGGLE CHANGE: Add scaling factors ---
        # The model output boxes are for a 256x256 image
        scale_x = orig_w / 256.0
        scale_y = orig_h / 256.0

        # Draw results on the image
        draw = ImageDraw.Draw(img_pil)
        for box, label_id, score in zip(boxes, labels, scores):
            
            # --- KAGGLE CHANGE: Scale the box ---
            x1, y1, x2, y2 = box
            scaled_box = [
                x1 * scale_x, 
                y1 * scale_y, 
                x2 * scale_x, 
                y2 * scale_y
            ]
            
            # Draw the *scaled* box
            draw.rectangle(list(scaled_box), outline="red", width=3)

            # Get class name (show_labels=True)
            class_name = COCO_CATEGORY_MAP.get(label_id, f'ID: {label_id}')

            # Create text (show_conf=True)
            text = f"{class_name} {score:.2f}"

            # Draw text above the *scaled* box
            text_position = (scaled_box[0], scaled_box[1] - 10)
            draw.text(text_position, text, fill="red", font=font)

        # Save the resulting image (save=True)
        base_filename = os.path.basename(img_path)
        save_path = os.path.join(OUTPUT_DIR, base_filename)
        img_pil.save(save_path)

    print("\n✅ Complete. Predictions with labels saved to:")
    print(OUTPUT_DIR)
    print("You can view the saved images in the Kaggle output viewer.")

# --- Run the main function ---
if __name__ == "__main__":
    run_inference_on_all_val()

Loading fine-tuned model...
Model loaded.
Running predictions on all 548 images in: /kaggle/working/coco_dataset/images/val/


100%|██████████| 548/548 [00:25<00:00, 21.74it/s]



✅ Complete. Predictions with labels saved to:
/kaggle/working/runs/detect/predictions_with_labels_FASTER_RCNN
You can view the saved images in the Kaggle output viewer.


In [9]:
import os
import torch
import glob
from PIL import Image, ImageDraw, ImageFont
from torchvision.transforms import v2 as T
from tqdm import tqdm

# --- KAGGLE CHANGE: Add imports and function needed to build the model ---
from torchvision.models.detection import (
    fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_faster_rcnn_model(num_classes):
    """
    Loads a pre-trained Faster R-CNN model and replaces the classifier head.
    """
    model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)
    # print("Using lightweight MobileNetV3-Large backbone.") # Silenced for less output
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model
# --- End of added function ---


# NOTE: We don't need VISDRONE_CATEGORIES since show_labels=False

# --- 2. Main Inference Function ---
def run_inference_on_all_val_boxes_only():
    print("Loading fine-tuned model...")
    # --- Paths and Parameters ---
    # --- KAGGLE CHANGE ---
    MODEL_DIR = '/kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN'
    MODEL_PATH = os.path.join(MODEL_DIR, 'faster_rcnn_final.pt')
    NUM_CLASSES = 11 # 10 VisDrone classes + 1 background

    # Equivalent to your YOLO script's parameters
    CONF_THRESHOLD = 0.25
    # --- KAGGLE CHANGE ---
    VAL_IMAGES_PATH = '/kaggle/working/coco_dataset/images/val/' # Use the COCO dataset path

    # Equivalent to project='runs/detect', name='predictions_boxes_only'
    # --- KAGGLE CHANGE ---
    OUTPUT_DIR = '/kaggle/working/runs/detect/predictions_boxes_only_FASTER_RCNN'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

    if not os.path.exists(MODEL_PATH) or not get_faster_rcnn_model:
        print(f"❌ ERROR: Model file not found at {MODEL_PATH} or 'get_faster_rcnn_model' not available.")
        print("Please ensure the training cell ran successfully.")
        return

    # --- Load Model ---
    model = get_faster_rcnn_model(NUM_CLASSES)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.to(device)
    model.eval() # Set model to evaluation mode
    print("Model loaded.")

    # --- Get all image paths ---
    val_images_paths = glob.glob(os.path.join(VAL_IMAGES_PATH, '*.jpg'))
    if not val_images_paths:
        print(f"No images found in {VAL_IMAGES_PATH}")
        return

    print(f"Running predictions (boxes only) on all {len(val_images_paths)} images in: {VAL_IMAGES_PATH}")

    # --- KAGGLE CHANGE: CRITICAL FIX ---
    # The transform MUST match the training transform, including the resize.
    transform = T.Compose([
        T.ToImage(),
        T.Resize((256, 256), antialias=True), # <-- This was missing
        T.ToDtype(torch.float32, scale=True)
    ])

    # --- Loop and predict (equivalent to model.predict) ---
    for img_path in tqdm(val_images_paths):
        # Load and transform the image
        img_pil = Image.open(img_path).convert('RGB')
        
        # --- KAGGLE CHANGE: Store original size for scaling boxes ---
        orig_w, orig_h = img_pil.size
        
        img_tensor = transform(img_pil) # This creates a 256x256 tensor
        input_batch = [img_tensor.to(device)]

        # Run prediction
        with torch.no_grad():
            predictions = model(input_batch)

        pred = predictions[0]

        # Filter boxes by confidence
        high_conf_indices = pred['scores'] > CONF_THRESHOLD
        boxes = pred['boxes'][high_conf_indices].cpu().numpy()
        # No need to get labels or scores since we aren't drawing them
        
        # --- KAGGLE CHANGE: Add scaling factors ---
        # The model output boxes are for a 256x256 image
        scale_x = orig_w / 256.0
        scale_y = orig_h / 256.0

        # Draw results on the image
        draw = ImageDraw.Draw(img_pil)
        for box in boxes:
            
            # --- KAGGLE CHANGE: Scale the box ---
            x1, y1, x2, y2 = box
            scaled_box = [
                x1 * scale_x, 
                y1 * scale_y, 
                x2 * scale_x, 
                y2 * scale_y
            ]

            # Draw the *scaled* box (show_labels=False, show_conf=False)
            draw.rectangle(list(scaled_box), outline="red", width=3)

            # --- All label/confidence drawing code is removed ---

        # Save the resulting image (save=True)
        base_filename = os.path.basename(img_path)
        save_path = os.path.join(OUTPUT_DIR, base_filename)
        img_pil.save(save_path)

    print("\n✅ Complete. Predictions (boxes only) saved to:")
    print(OUTPUT_DIR)
    print("You can view the saved images in the Kaggle output viewer.")

# --- Run the main function ---
if __name__ == "__main__":
    run_inference_on_all_val_boxes_only()

Loading fine-tuned model...
Model loaded.
Running predictions (boxes only) on all 548 images in: /kaggle/working/coco_dataset/images/val/


100%|██████████| 548/548 [00:15<00:00, 35.97it/s]


✅ Complete. Predictions (boxes only) saved to:
/kaggle/working/runs/detect/predictions_boxes_only_FASTER_RCNN
You can view the saved images in the Kaggle output viewer.


In [10]:
import shutil
import os

# --- KAGGLE CHANGE ---
# All paths must point to /kaggle/working/
base_runs_dir = '/kaggle/working/runs/detect'

# --- KAGGLE CHANGE ---
# All code related to 'drive_save_path' has been removed.
# Zip files will be created in /kaggle/working/
# You can download them from the "Output" tab in the Kaggle UI.

# --- 1. Zip the Main Training Run (includes .pt model files) ---
folder_to_zip = 'finetune_visdrone_night_scaled_FASTER_RCNN'
zip_name = 'finetune_visdrone_night_scaled_FASTER_RCNN_RUN'
full_folder_path = os.path.join(base_runs_dir, folder_to_zip)

if os.path.exists(full_folder_path):
    print(f"Zipping {folder_to_zip}...")
    shutil.make_archive(
        zip_name,
        'zip',
        root_dir=base_runs_dir,
        base_dir=folder_to_zip
    )
    print(f"✅ Created {zip_name}.zip in /kaggle/working/")

    # --- KAGGLE CHANGE ---
    # Removed 'Move to Drive' block
else:
    print(f"⚠️ Warning: Folder not found, skipping zip: {full_folder_path}")


# --- 2. Zip the Predictions (With Labels) ---
folder_to_zip = 'predictions_with_labels_FASTER_RCNN'
zip_name_labels = 'predictions_with_labels_FASTER_RCNN_IMAGES'
full_folder_path = os.path.join(base_runs_dir, folder_to_zip)

if os.path.exists(full_folder_path):
    print(f"\nZipping {folder_to_zip}...")
    shutil.make_archive(
        zip_name_labels,
        'zip',
        root_dir=base_runs_dir,
        base_dir=folder_to_zip
    )
    print(f"✅ Created {zip_name_labels}.zip in /kaggle/working/")

    # --- KAGGLE CHANGE ---
    # Removed 'Move to Drive' block
else:
    print(f"⚠️ Warning: Folder not found, skipping zip: {full_folder_path}")


# --- 3. Zip the Predictions (Boxes Only) ---
folder_to_zip = 'predictions_boxes_only_FASTER_RCNN'
zip_name_boxes = 'predictions_boxes_only_FASTER_RCNN_IMAGES'
full_folder_path = os.path.join(base_runs_dir, folder_to_zip)

if os.path.exists(full_folder_path):
    print(f"\nZipping {folder_to_zip}...")
    shutil.make_archive(
        zip_name_boxes,
        'zip',
        root_dir=base_runs_dir,
        base_dir=folder_to_zip
    )
    print(f"✅ Created {zip_name_boxes}.zip in /kaggle/working/")

    # --- KAGGLE CHANGE ---
    # Removed 'Move to Drive' block
else:
    print(f"⚠️ Warning: Folder not found, skipping zip: {full_folder_path}")


print("---" * 20)
print("✅ ALL DONE. Your Faster R-CNN training run, model, and prediction images have been saved to /kaggle/working/.")
print("You can download the .zip files from the 'Output' section in the Kaggle UI.")
print("---" * 20)

Zipping finetune_visdrone_night_scaled_FASTER_RCNN...
✅ Created finetune_visdrone_night_scaled_FASTER_RCNN_RUN.zip in /kaggle/working/

Zipping predictions_with_labels_FASTER_RCNN...
✅ Created predictions_with_labels_FASTER_RCNN_IMAGES.zip in /kaggle/working/

Zipping predictions_boxes_only_FASTER_RCNN...
✅ Created predictions_boxes_only_FASTER_RCNN_IMAGES.zip in /kaggle/working/
------------------------------------------------------------
✅ ALL DONE. Your Faster R-CNN training run, model, and prediction images have been saved to /kaggle/working/.
You can download the .zip files from the 'Output' section in the Kaggle UI.
------------------------------------------------------------


In [25]:
# --- CELL: EVALUATE mAP@[.5:.05:.95] (MANUAL PYTHON/TORCH) ---

import os
import torch
import torchvision
import json
from torch.utils.data import DataLoader
from torchvision.transforms import v2 as T
from tqdm import tqdm
import numpy as np

# --- 1. Redefine Dataset and Model Functions ---
# (Must be redefined in case the session was reset or to run independently)

from torchvision.models.detection import (
    fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

class CustomCocoDataset(torchvision.datasets.CocoDetection):
    def __init__(self, root, annFile, transform=None):
        super(CustomCocoDataset, self).__init__(root, annFile)
        self.transform = transform
        # We also need to load the raw COCO ground truth
        from pycocotools.coco import COCO
        self.coco = COCO(annFile)
        # Fix for the 'info' key error
        if 'info' not in self.coco.dataset:
            self.coco.dataset['info'] = {}

    def __getitem__(self, idx):
        img, target_list = super(CustomCocoDataset, self).__getitem__(idx)
        
        if self.transform is not None:
            img = self.transform(img)

        target = {}
        image_id = self.ids[idx]
        target["image_id"] = torch.tensor([image_id])
        
        # Get original image info
        img_info = self.coco.loadImgs(image_id)[0]
        target["orig_w"] = img_info['width']
        target["orig_h"] = img_info['height']
        
        # Get ground truth boxes
        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        anns = self.coco.loadAnns(ann_ids)
        
        gt_boxes = []
        gt_labels = []
        for ann in anns:
            # Skip 'iscrowd' annotations
            if ann.get('iscrowd', 0) == 1:
                continue
            # VisDrone class 0 and 11 are ignored
            if ann['category_id'] in [0, 11]:
                continue
            
            x, y, w, h = ann['bbox']
            gt_boxes.append([x, y, x + w, y + h]) # to [x1, y1, x2, y2]
            gt_labels.append(ann['category_id'])

        if not gt_boxes:
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros(0, dtype=torch.int64)
        else:
            target["boxes"] = torch.as_tensor(gt_boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(gt_labels, dtype=torch.int64)

        return img, target

def get_faster_rcnn_model(num_classes):
    model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def collate_fn(batch):
    return tuple(zip(*batch))

# --- 2. Define IoU Helper Function ---
def calculate_iou(box1, box2):
    """Calculates IoU for two [x1, y1, x2, y2] boxes"""
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union_area = box1_area + box2_area - inter_area
    
    if union_area == 0:
        return 0.0
    return inter_area / union_area

# --- 3. Setup Paths and Parameters ---
MODEL_PATH = '/kaggle/working/runs/detect/finetune_visdrone_night_scaled_FASTER_RCNN/faster_rcnn_final.pt'
VAL_IMG_DIR = '/kaggle/working/coco_dataset/images/val'
VAL_ANN_FILE = '/kaggle/working/coco_dataset/annotations/instances_val.json'
NUM_CLASSES = 11 # 10 classes + 1 background
BATCH_SIZE = 16
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

# This is [0.5, 0.55, 0.6, ..., 0.90, 0.95]
IOU_THRESHOLDS = np.linspace(0.5, 0.95, 10)
print(f"Calculating mAP for 10 IoU thresholds: {IOU_THRESHOLDS}")
print(f"Using device: {device}")

# --- 4. Load Data and Model ---
val_transform = T.Compose([
    T.ToImage(), T.Resize((256, 256), antialias=True), T.ToDtype(torch.float32, scale=True)
])
dataset_val = CustomCocoDataset(root=VAL_IMG_DIR, annFile=VAL_ANN_FILE, transform=val_transform)
data_loader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, collate_fn=collate_fn)

print("Loading model for evaluation...")
model = get_faster_rcnn_model(NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()
print("Model loaded.")

# --- 5. Evaluation Loop (Run Once) ---
all_preds = [] # List of [image_idx, class_id, score, x1, y1, x2, y2]
all_gts = []   # List of [image_idx, class_id, x1, y1, x2, y2]

print(f"Running evaluation on {len(dataset_val)} validation images...")

with torch.no_grad():
    for batch_idx, (images, targets) in enumerate(tqdm(data_loader_val, desc="Evaluating")):
        images = list(img.to(device) for img in images)
        predictions = model(images)
        
        for i, (pred, target) in enumerate(zip(predictions, targets)):
            image_idx = target["image_id"].item()
            
            # --- Store Ground Truths ---
            for gt_box, gt_label in zip(target['boxes'], target['labels']):
                all_gts.append([image_idx, gt_label.item()] + gt_box.tolist())

            # --- Store Predictions ---
            orig_w, orig_h = target['orig_w'], target['orig_h']
            scale_x, scale_y = orig_w / 256.0, orig_h / 256.0
            
            for box, label, score in zip(pred['boxes'], pred['labels'], pred['scores']):
                x1, y1, x2, y2 = box.tolist()
                scaled_box = [x1 * scale_x, y1 * scale_y, x2 * scale_x, y2 * scale_y]
                all_preds.append([image_idx, label.item(), score.item()] + scaled_box)

print("Evaluation loop complete. Calculating mAP...")

# --- 6. Calculate mAP for all IoU Thresholds ---
all_map_scores = [] # Store the mAP for each IoU threshold

for iou_threshold in IOU_THRESHOLDS:
    average_precisions = []
    # VisDrone categories are 1 through 10
    for class_id in range(1, 11): 
        
        class_preds = sorted([p for p in all_preds if p[1] == class_id], key=lambda x: x[2], reverse=True)
        class_gts = [g for g in all_gts if g[1] == class_id]
        
        total_gts_for_class = len(class_gts)
        if total_gts_for_class == 0:
            continue 

        # --- 🔴 LOGIC FIX HERE ---
        # Create a dictionary to track detected GTs *per image*
        # Key: image_idx, Value: list of bools (one for each GT box *in that image*)
        gt_detected_map = {}
        for g in class_gts:
            img_idx = g[0]
            if img_idx not in gt_detected_map:
                # Count how many GTs for this class are in this image
                gts_in_img = len([gt for gt in class_gts if gt[0] == img_idx])
                gt_detected_map[img_idx] = [False] * gts_in_img
        # --- End of Fix ---

        true_positives = np.zeros(len(class_preds))
        false_positives = np.zeros(len(class_preds))
        
        for pred_idx, pred in enumerate(class_preds):
            img_idx = pred[0]
            pred_box = pred[3:]
            
            # --- 🔴 LOGIC FIX HERE ---
            # Get only GTs for this class *in this image*
            # Store them with their *local index* for this image
            gts_for_this_image = [g[2:] for g in class_gts if g[0] == img_idx]
            gt_boxes_for_img = list(enumerate(gts_for_this_image))
            # --- End of Fix ---
            
            if not gt_boxes_for_img:
                false_positives[pred_idx] = 1
                continue

            best_iou = 0
            best_gt_local_idx = -1 # This is the index *within the image's list*
            
            # Find the best matching ground truth box
            for local_idx, gt_box in gt_boxes_for_img:
                iou = calculate_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_local_idx = local_idx # This is now the correct LOCAL index (0, 1, ...)
            
            if best_iou >= iou_threshold:
                # --- 🔴 LOGIC FIX HERE ---
                # Use the local_idx to check the detection map
                if not gt_detected_map[img_idx][best_gt_local_idx]:
                    true_positives[pred_idx] = 1
                    gt_detected_map[img_idx][best_gt_local_idx] = True
                else:
                    false_positives[pred_idx] = 1
                # --- End of Fix ---
            else:
                false_positives[pred_idx] = 1
                
        # Calculate precision and recall
        tp_cumsum = np.cumsum(true_positives)
        fp_cumsum = np.cumsum(false_positives)
        
        precision = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-16)
        recall = tp_cumsum / (total_gts_for_class + 1e-16)
        
        # Calculate Average Precision (AP)
        precision = np.concatenate(([0.], precision, [0.]))
        recall = np.concatenate(([0.], recall, [1.]))
        
        for i in range(len(precision) - 2, -1, -1):
            precision[i] = max(precision[i], precision[i + 1])
            
        recall_changes = np.where(recall[1:] != recall[:-1])[0] + 1
        
        ap = np.sum((recall[recall_changes] - recall[recall_changes - 1]) * precision[recall_changes])
        average_precisions.append(ap)
    
    # Store the mAP for this IoU threshold
    map_at_iou = np.mean(average_precisions)
    all_map_scores.append(map_at_iou)
    # Print the result for this threshold
    print(f"  -> mAP @ IoU={iou_threshold:.2f} = {map_at_iou:.4f}")


# --- 7. Print Final Results ---
final_map = np.mean(all_map_scores)

print("\n--- FASTER R-CNN mAP RESULTS (Manual) ---")
print(f"mAP (IoU=0.50): {all_map_scores[0]:.4f}")
print(f"mAP (IoU=0.75): {all_map_scores[5]:.4f}")
print("-------------------------------------------")
print(f"mAP@[.5:.05:.95]: {final_map:.4f}")
print("-------------------------------------------")

Calculating mAP for 10 IoU thresholds: [        0.5        0.55         0.6        0.65         0.7        0.75         0.8        0.85         0.9        0.95]
Using device: cuda:0
loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
loading annotations into memory...
Done (t=0.43s)
creating index...
index created!
Loading model for evaluation...
Model loaded.
Running evaluation on 548 validation images...


Evaluating: 100%|██████████| 35/35 [00:16<00:00,  2.17it/s]


Evaluation loop complete. Calculating mAP...
  -> mAP @ IoU=0.50 = 0.0387
  -> mAP @ IoU=0.55 = 0.0339
  -> mAP @ IoU=0.60 = 0.0288
  -> mAP @ IoU=0.65 = 0.0227
  -> mAP @ IoU=0.70 = 0.0183
  -> mAP @ IoU=0.75 = 0.0126
  -> mAP @ IoU=0.80 = 0.0078
  -> mAP @ IoU=0.85 = 0.0033
  -> mAP @ IoU=0.90 = 0.0005
  -> mAP @ IoU=0.95 = 0.0000

--- FASTER R-CNN mAP RESULTS (Manual) ---
mAP (IoU=0.50): 0.0387
mAP (IoU=0.75): 0.0126
-------------------------------------------
mAP@[.5:.05:.95]: 0.0167
-------------------------------------------
